In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/lycoris : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Default = 'https://civitai.com/api/download/models/107236?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_DefaultName = 'yabaLMix_v5.safetensors' #@param {type:"string"}

#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = True #@param {type:"boolean"}

Workspace = 'comfyui'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lora
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lycoris
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/embedding
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/wildcards

checkpoint_url = Checkpoint_Default
checkpoint_name = Checkpoint_DefaultName
checkpoint_down_path = f'/content/{Workspace}/models/checkpoints'
if Google_Drive and Checkpoint_Google_Save:
  checkpoint_down_path = f'/content/{Workspace}/models/checkpoints'

checkpoint_file = checkpoint_down_path+'/'+checkpoint_name

%cd /content

!git clone https://github.com/comfyanonymous/ComfyUI /content/{Workspace}
%cd {Workspace}

!apt -y install -qq aria2
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

# 구글드라이브 연결
if Google_Drive:
  !mkdir /content/{workspace}/models/loras
  !mkdir /content/{Workspace}/models/hypernetworks
  !rm /content/{Workspace}/models/embeddings/google
  !rm /content/{Workspace}/models/checkpoints/google
  !rm /content/{Workspace}/models/loras/google
  !rm /content/{Workspace}/models/hypernetworks/google
  !rm /content/{Workspace}/output
  !mv /content/{Workspace}/output /content/{Workspace}/output_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/embedding /content/{Workspace}/models/embeddings/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint /content/{Workspace}/models/checkpoints/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lora /content/{Workspace}/models/loras/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork /content/{Workspace}/models/hypernetworks/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/{Workspace}/output

# controlnet models
if ControlNet:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15s2_lineart_anime_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11f1e_sd15_tile_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/{Workspace}/models/controlnet -o control_v11p_sd15_scribble_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d /content/{Workspace}/models/controlnet -o diff_control_sd15_temporalnet_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d /content/{Workspace}/models/controlnet -o t2iadapter_style_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d /content/{Workspace}/models/controlnet -o t2iadapter_color_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d /content/{Workspace}/models/controlnet -o t2iadapter_sketch_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d /content/{Workspace}/models/controlnet -o t2iadapter_zoedepth_sd15v1.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{checkpoint_url}" -d {checkpoint_down_path} -o {checkpoint_name}

# upscaler
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /content/{Workspace}/models/upscale_models/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/upscale_models -o 4x-UltraSharp.pth

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/{Workspace}/models/vae -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/{Workspace}/models/vae -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/loras -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" -d /content/{Workspace}/models/loras -o LowRA.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/models/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/models/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/{Workspace}/models/embeddings -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/models/embeddings -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/models/embeddings -o bad_prompt_version2.pt

# Run ComfyUI with localtunnel (Recommended Way)
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server